In [10]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Define the indices and their corresponding tickers
indices = {
    "标普500": "^GSPC",
    "道琼斯": "^DJI",
    "纳斯达克": "^IXIC",
    "罗素2000": "^RUT",
    "VIX": "^VIX"
}

# Get the latest data and historical data for each index
data = []
today = datetime.now()
one_week_ago = today - timedelta(days=7)

for name, ticker in indices.items():
    index = yf.Ticker(ticker)
    hist = index.history(period="1mo")
    
    # Ensure the index is timezone-naive
    hist.index = hist.index.tz_localize(None)
    
    # Get the latest closing price
    latest_close = hist['Close'].iloc[-1]
    latest_date = hist.index[-1].date()
    
    # Get the previous day's closing price
    prev_close = hist['Close'].iloc[-2]

    # Convert one_week_ago to a Timestamp and ensure it's timezone-naive
    one_week_ago_ts = pd.Timestamp(one_week_ago).tz_localize(None)

    # Get the closing price from one week ago (if available)
    one_week_close = hist['Close'].asof(one_week_ago_ts)
    
    # Calculate percentage changes
    daily_change = ((latest_close - prev_close) / prev_close) * 100
    weekly_change = ((latest_close - one_week_close) / one_week_close) * 100 if one_week_close else None

    data.append({
        "指数名称": name,
        "最新收盘": round(latest_close, 2),
        "日期": latest_date,
        "较前日涨跌 (%)": round(daily_change, 2),
        "较一周前涨跌 (%)": round(weekly_change, 2) if weekly_change is not None else "N/A"
    })

# Convert to DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print("最新指数数据对比：")
print(df)


最新指数数据对比：
     指数名称      最新收盘          日期  较前日涨跌 (%)  较一周前涨跌 (%)
0   标普500   5955.25  2025-02-25      -0.47       -2.84
1     道琼斯  43621.16  2025-02-25       0.37       -2.10
2    纳斯达克  19026.39  2025-02-25      -1.35       -5.06
3  罗素2000   2170.08  2025-02-25      -0.38       -5.25
4     VIX     19.43  2025-02-25       2.37       26.58
